# Good Morning(早安)

# Preface




### (just for those guy who haven't used python/colab)

1.   (for colab newcomer) you just need to continuously press "shift+enter"
2.   (for python newcomer) Python is a Interpreted Language unlike C, so you can just only run a part of codes (which don't need compile all of them at fisrt).

# Game start

## Markov Chain?
What is markov chain?

> In probability theory and statistics, a Markov chain or Markov process is a stochastic process describing a sequence of possible events in which the probability of each event depends only on the state attained in the previous event. Informally, this may be thought of as, "What happens next depends only on the state of affairs now." A countably infinite sequence, in which the chain moves state at discrete time steps, gives a discrete-time Markov chain (DTMC). A continuous-time process is called a continuous-time Markov chain (CTMC). Markov processes are named in honor of the Russian mathematician Andrey Markov.

---By wiki zzz...


Simply, actually we learn it in high school already, it's just about **state transfer**. We concern about what the next state should be according to the state now.

## How can it use on Making Music?
Actually, the idea is simple. We random find a musical note as beginning. Then according to markov chain we computed, we make next and next and next.....

Then, it's a music based on the original style.zzz.

## Detailed Operation about Markov Chain
Let's deeply discuss about how a markov chain generate based on a song (or a better way to say, a lots of notes). Actually, it's easy, too. For example, what is the probability of we tranfer note "G4"(so) to "E4"(mi)? Simplly, it's the time "E4" appear after "G4" divide by the time "G4" appear.

Also, a music consist not only by note but also need note duration. So, I do the same thing on it to create another markov chain.

In [ ]:
# install the library we need
!pip3 install midiutil
!sudo apt-get install fluidsynth
!pip3 install midi2audio
!pip3 install pydub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for midiutil: filename=MIDIUtil-1.2.1-py3-none-any.whl size=54569 sha256=4309f6a4a259cc04168aeb625b61f0db042aa66c3618a594dca2ceac443d4d47
  Stored in directory: /root/.cache/pip/wheels/af/43/4a/00b5e4f2fe5e2cd6e92b461995a3a97a2cebb30ab5783501b0
Successfully built midiutil
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fluid-soundfont-gm libevdev2 libfluidsynth3 libgudev-1.0-0 libinput-bin
  libinput10 libinstpatch-1.0-2 libmd4c0 libmtdev1 libqt5core5a libqt5dbus5
  libqt5gui5 libqt5network5 libqt5svg5 libqt5widgets5 libwacom-bin
  libwacom-common libwacom9 libxcb-icccm4 libxcb-image0 libxcb-keysyms1
  libxcb-render-util0 libxcb-util1 libxcb-xinerama0 libxcb-xinput0 libxcb-xkb1
  libxkbcommon-x11-0 qsynth qt5-gtk-platformtheme qttranslations5-l10n
  

In [ ]:
# import the library we need
from midiutil import MIDIFile
import librosa
from collections import defaultdict
import random
from midi2audio import FluidSynth
from pydub import AudioSegment
import os

In [ ]:
# this is the original song's note and note duration
# you can change to any song you like
original_melody = [
    ("G4", 0.5), ("E4", 0.5), ("E4", 1.0), ("F4", 0.5), ("D4", 0.5), ("D4", 1.0),
    ("C4", 0.5), ("D4", 0.5), ("E4", 0.5), ("F4", 0.5), ("G4", 0.5), ("G4", 0.5), ("G4", 1.0)
]

In [ ]:
# build markov chain for note and note duration
def build_markov_chain_for_notes_and_durations(melody):
    note_chain = defaultdict(lambda: defaultdict(int))
    duration_chain = defaultdict(lambda: defaultdict(int))

    for i in range(1, len(melody)):
        prev_note, prev_duration = melody[i-1]
        current_note, current_duration = melody[i]

        note_chain[prev_note][current_note] += 1
        duration_chain[prev_duration][current_duration] += 1

    for prev_note in note_chain:
        total = sum(note_chain[prev_note].values())
        for next_note in note_chain[prev_note]:
            note_chain[prev_note][next_note] /= total

    for prev_duration in duration_chain:
        total = sum(duration_chain[prev_duration].values())
        for next_duration in duration_chain[prev_duration]:
            duration_chain[prev_duration][next_duration] /= total

    return note_chain, duration_chain

# I originally wanted to write something, but I found it to be very intuitive. zzz.
# Just like what I say, compute the time of all the note to the note and the time of its total apperance.

In [ ]:
def generate_melody(note_chain, duration_chain, start_note, start_duration, num_notes=10):
    # (for those who haven't learnd python) Unlike C/C++, python don't need to claim a enough length for an array first.
    # It can directly use arr.append("the_thing_you_want_to_insert") to append something.
    generated_melody = [(start_note, start_duration)]
    current_note = start_note
    current_duration = start_duration

    for _ in range(num_notes - 1):
        # According to the probability, random choice
        if current_note in note_chain:
            next_note = random.choices(
                list(note_chain[current_note].keys()),
                weights=list(note_chain[current_note].values()),
                k=1
            )[0]
        else:
            next_note = random.choice(list(note_chain.keys()))

        # According to the probability, random choice
        if current_duration in duration_chain:
            next_duration = random.choices(
                list(duration_chain[current_duration].keys()),
                weights=list(duration_chain[current_duration].values()),
                k=1
            )[0]
        else:
            next_duration = random.choice(list(duration_chain.keys()))

        generated_melody.append((next_note, next_duration))
        # renew the next to current
        current_note = next_note
        current_duration = next_duration

    return generated_melody

In [ ]:
# To tranfer to the .mp3
# This part is unimportant. You guys can ignore it.
def frequencies_to_midi(frequencies, output_file):
    midi = MIDIFile(1)
    track = 0
    time = 0
    midi.addTrackName(track, time, "Track")
    midi.addTempo(track, time, 120)

    for note, duration in frequencies:
        midi_note = int(librosa.hz_to_midi(librosa.note_to_hz(note)))
        midi_note = max(0, min(127, midi_note))
        midi.addNote(track, 0, midi_note, time, duration, 100)
        time += duration

    with open(output_file, 'wb') as outf:
        midi.writeFile(outf)


def midi_to_mp3(midi_file_path, mp3_file_path):
    fs = FluidSynth()
    wav_file_path = "temp_audio.wav"
    fs.midi_to_audio(midi_file_path, wav_file_path)

    audio = AudioSegment.from_wav(wav_file_path)
    audio.export(mp3_file_path, format="mp3")
    os.remove("temp_audio.wav")
    # os.remove(midi_file_path)

In [ ]:
note_chain, duration_chain = build_markov_chain_for_notes_and_durations(original_melody)

# If you want to listen more, you can adjust the "num_notes" by yourself. It's about the number of note it generate.
generated_melody = generate_melody(note_chain, duration_chain, "G4", 0.5, num_notes=20)

frequencies_to_midi(generated_melody, "generated_song.mid")
# You can find the file in the left if you want to download. zzz.

In [ ]:
# Listen directly!!!
midi_file_path = "generated_song.mid"
mp3_file_path = "output_music.mp3"
midi_to_mp3(midi_file_path, mp3_file_path)
from IPython.display import Audio


Audio("output_music.mp3", autoplay=True)

# Game Over

## Why you still here, the game is over!!!

Alright. The fllowing the code is not about generating music. It's just for me to test whether I input the correct small bee's notes (yes, this is the most difficult part for me).

So this part is trying to tranfer notes and note durations to real music. zzz...

### 如果你想聽小蜜蜂(If you want to listen small bee. zzz.)


In [ ]:
from midiutil import MIDIFile
from pydub import AudioSegment
from midi2audio import FluidSynth

In [ ]:
def melody_to_midi(melody, output_file):
    midi = MIDIFile(1)
    track = 0
    time = 0
    midi.addTrackName(track, time, "Track")
    midi.addTempo(track, time, 120)

    for note, duration in melody:
        midi_note = note_to_midi(note)
        midi.addNote(track, 0, midi_note, time, duration, 100)
        time += duration

    with open(output_file, 'wb') as outf:
        midi.writeFile(outf)

def note_to_midi(note):
    note_dict = {
        "C4": 60, "D4": 62, "E4": 64, "F4": 65, "G4": 67,
        "A4": 69, "B4": 71, "C5": 72, "D5": 74, "E5": 76,
        "F5": 77, "G5": 79, "A5": 81, "B5": 83
    }
    return note_dict.get(note, 60)

def midi_to_mp3(midi_file, mp3_file):
    fs = FluidSynth()
    fs.midi_to_audio(midi_file, "temp.wav")


    sound = AudioSegment.from_wav("temp.wav")
    sound.export(mp3_file, format="mp3")
    os.remove("temp_audio.wav")
    os.remove(midi_file_path)

In [ ]:
melody_to_midi(original_melody, "generated_song.mid")
midi_to_mp3("generated_song.mid", "output_music.mp3")